# 0.0 IMPORTS

## 01. Libraries

In [5]:
import warnings
import inflection
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt

import dask.dataframe as dd

from dask.distributed import Client, progress

In [21]:
# import datetime
# import numpy as np
# import scipy.stats  as ss

# from IPython.display         import Image
# from IPython.core.display    import HTML

# import matplotlib.pyplot as plt
# import seaborn as sns

# 0.2 Notebook Config

In [2]:
warnings.filterwarnings("ignore")

In [3]:
client = Client()
client

Client Scheduler: tcp://127.0.0.1:54192 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 4 Cores: 12 Memory: 16.98 GB


In [22]:
# def jupyter_settings():
#     %matplotlib inline
    
#     plt.style.use('bmh')
#     plt.rcParams['figure.figsize'] = [25, 12]
#     plt.rcParams['font.size'] = 24
    
#     display( HTML('<style>.container { width:100% !important; }</style>'))
    
#     warnings.filterwarnings("ignore")
    
#     sns.set()

In [23]:
# seed = 42
# np.random.seed(seed)

# jupyter_settings()

## 0.3 Helper Functions

## 0.4 Load Data

### 0.4.1 Load Raw Data

#### 0.4.1.1 train

In [6]:
types = {'Semana':np.uint8, 'Agencia_ID':np.uint16, 'Canal_ID':np.uint8,
         'Ruta_SAK':np.uint16, 'Cliente_ID':np.uint32, 'Producto_ID':np.uint16,
         'Demanda_uni_equil':np.uint32}

data_train_raw = dd.read_csv('../00-Data/csv_data/train.csv', usecols=types.keys(), dtype=types)

In [7]:
types = {'Producto_ID':np.uint16,
         'NombreProducto':'object'}


data_producto_raw = dd.read_csv('../00-Data/csv_data/producto_tabla.csv', usecols=types.keys(), dtype=types)

In [8]:
types = {'Cliente_ID':np.uint32,
         'NombreCliente':'object'}


data_cliente_raw = dd.read_csv('../00-Data/csv_data/cliente_tabla.csv', usecols=types.keys(), dtype=types)

In [9]:
types = {'Agencia_ID':np.uint16,
         'Town':'object',
         'State':'object'}


data_town_state_raw = dd.read_csv('../00-Data/csv_data/town_state.csv', usecols=types.keys(), dtype=types)

In [10]:
# data_train_raw = dd.read_csv('../00-Data/csv_data/train.csv')
# data_producto_raw = dd.read_csv('../00-Data/csv_data/producto_tabla.csv')
# data_cliente_raw = dd.read_csv('../00-Data/csv_data/cliente_tabla.csv')
# data_town_state_raw = dd.read_csv('../00-Data/csv_data/town_state.csv')

### 0.4.2 Merge Dataset

In [11]:
data_raw = dd.merge(data_train_raw, data_producto_raw, how='left', on='Producto_ID')
data_raw = dd.merge(data_raw, data_cliente_raw, how='left', on='Cliente_ID')
data_raw = dd.merge(data_raw, data_town_state_raw, how='left', on='Agencia_ID')
data_raw = data_raw.drop(['Producto_ID', 'Cliente_ID', 'Agencia_ID'], axis=1)
data_raw.head()

,Semana,Canal_ID,Ruta_SAK,Demanda_uni_equil,NombreProducto,NombreCliente,Town,State
0,3,7,3301,3,Roles Canela 2p 120g BIM 1212,PUESTO DE PERIODICOS LAZARO,2008 AG. LAGO FILT,"MÉXICO, D.F."
1,3,7,3301,4,Roles Glass 2p 135g BIM 1216,PUESTO DE PERIODICOS LAZARO,2008 AG. LAGO FILT,"MÉXICO, D.F."
2,3,7,3301,4,Panquecito Gota Choc 2p 140g BIM 1238,PUESTO DE PERIODICOS LAZARO,2008 AG. LAGO FILT,"MÉXICO, D.F."
3,3,7,3301,4,Mantecadas Vainilla 4p 125g BIM 1240,PUESTO DE PERIODICOS LAZARO,2008 AG. LAGO FILT,"MÉXICO, D.F."
4,3,7,3301,3,Donitas Espolvoreadas 6p 105g BIM 1242,PUESTO DE PERIODICOS LAZARO,2008 AG. LAGO FILT,"MÉXICO, D.F."


# 1.0 DATA DESCRIPTION

In [12]:
df01 = data_raw.copy()

## 1.1 Columns

In [13]:
df01.columns

Index(['Semana', 'Canal_ID', 'Ruta_SAK', 'Demanda_uni_equil', 'NombreProducto',
       'NombreCliente', 'Town', 'State'],
      dtype='object')

### 1.2.1 Rename Columns

In [14]:
snakecase = lambda col: inflection.underscore(col)

new_columns = list(map(snakecase, df01.columns))

# rename
df01.columns = new_columns

### 1.2.2 New Name Columns

In [15]:
df01.columns

Index(['semana', 'canal_id', 'ruta_sak', 'demanda_uni_equil',
       'nombre_producto', 'nombre_cliente', 'town', 'state'],
      dtype='object')

## 1.3 Data Dimensions

In [16]:
print(f'Number of Rows: {df01.shape[0].compute()}')
print(f'Number of Columns: {df01.shape[1]}')

Number of Rows: 74773833
Number of Columns: 8


## 1.4 Data Types

In [17]:
df01.dtypes

semana                uint8
canal_id              uint8
ruta_sak             uint16
demanda_uni_equil    uint32
nombre_producto      object
nombre_cliente       object
town                 object
state                object
dtype: object

## 1.5 Missing Values

In [18]:
data_raw.isnull().sum().compute()

Semana               0
Canal_ID             0
Ruta_SAK             0
Demanda_uni_equil    0
NombreProducto       0
NombreCliente        0
Town                 0
State                0
dtype: int64

### 1.5.2 Fillout NA

There is no NA values

## 1.6 Descriptive Statistical

In [19]:
num_attributes = df01.select_dtypes(exclude=['object'])

In [ ]:
describe = num_attributes.describe().compute().T

describe['Relative Std'] = describe['std'] / describe['mean']
describe['range'] = describe['max'] - describe['min']
describe['IQR'] = describe['75%'] - describe['25%']
describe = describe[['min', 'max', 'range', 'mean','std', '50%', 'IQR', '25%', '75%', 'Relative Std']]
describe.columns = ['Min', 'Max', 'Range', 'Mean', 'Std', 'Median', 'IQR', '25%', '75%', 'Relative Std']
describe